In [130]:
import pandas as pd
import psycopg2
from sqlalchemy import create_engine
import warnings
warnings.filterwarnings("ignore")

In [2]:
!pip install psycopg2-binary


In [ ]:
##Psycopg2 connector

In [135]:
# DB connection setup using hardcoded credentials (for onboarding only)
conn = psycopg2.connect(
    dbname="neondb",
    user="neondb_owner",
    password="npg_CeS9fJg2azZD",
    host="ep-falling-glitter-a5m0j5gk-pooler.us-east-2.aws.neon.tech",
    port="5432",
    sslmode="require"  ##ssl encryption on
)
cur = conn.cursor()


In [105]:
## test query
query = "SELECT * FROM nyc_schools.high_school_directory LIMIT 5;"
df = pd.read_sql(query, conn)
df.head()

,dbn,school_name,borough,building_code,phone_number,fax_number,grade_span_min,grade_span_max,expgrade_span_min,expgrade_span_max,...,number_programs,Location 1,Community Board,Council District,Census Tract,Zip Codes,Community Districts,Borough Boundaries,City Council Districts,Police Precincts
0,27Q260,Frederick Douglass Academy VI High School,Queens,Q465,718-471-2154,718-471-2890,9.0,12,None,None,...,1,"{'latitude': '40.601989336', 'longitude': '-73...",14,31,100802,20529,51,3,47,59
1,21K559,Life Academy High School for Film and Music,Brooklyn,K400,718-333-7750,718-333-7775,9.0,12,None,None,...,1,"{'latitude': '40.593593811', 'longitude': '-73...",13,47,306,17616,21,2,45,35
2,16K393,Frederick Douglass Academy IV Secondary School,Brooklyn,K026,718-574-2820,718-574-2821,9.0,12,None,None,...,1,"{'latitude': '40.692133704', 'longitude': '-73...",3,36,291,18181,69,2,49,52
3,08X305,Pablo Neruda Academy,Bronx,X450,718-824-1682,718-824-1663,9.0,12,None,None,...,1,"{'latitude': '40.822303765', 'longitude': '-73...",9,18,16,11611,58,5,31,26
4,03M485,Fiorello H. LaGuardia High School of Music & A...,Manhattan,M485,212-496-0700,212-724-5748,9.0,12,None,None,...,6,"{'latitude': '40.773670507', 'longitude': '-73...",7,6,151,12420,20,4,19,12


In [6]:
## checking tables in db
query="""
SELECT table_schema, table_name
FROM information_schema.tables
WHERE table_type = 'BASE TABLE'
  AND table_schema NOT IN ('pg_catalog', 'information_schema')
ORDER BY table_schema, table_name;"""
df = pd.read_sql(query, conn)
df


,table_schema,table_name
0,nyc_schools,high_school_directory
1,nyc_schools,school_demographics
2,nyc_schools,school_safety_report


In [7]:
## checking columns
query="""
SELECT 
    table_schema, 
    table_name, 
    column_name, 
    data_type
FROM information_schema.columns
WHERE table_name = 'high_school_directory'
ORDER BY ordinal_position;"""
df = pd.read_sql(query, conn)
df

,table_schema,table_name,column_name,data_type
0,nyc_schools,high_school_directory,dbn,text
1,nyc_schools,high_school_directory,school_name,text
2,nyc_schools,high_school_directory,borough,text
3,nyc_schools,high_school_directory,building_code,text
4,nyc_schools,high_school_directory,phone_number,text
...,...,...,...,...
100,nyc_schools,high_school_directory,Zip Codes,character varying
101,nyc_schools,high_school_directory,Community Districts,character varying
102,nyc_schools,high_school_directory,Borough Boundaries,character varying
103,nyc_schools,high_school_directory,City Council Districts,character varying


In [25]:
# Clean column names (lowercase, strip, replace spaces with underscores)

query=f"""
SELECT * FROM nyc_schools.high_school_directory;"""
df = pd.read_sql(query, conn)
new_col_names=[]
for col in df.columns:
        stripped_name=col.lower().strip().replace(' ','_')
        new_col_names.append(stripped_name)
df.columns=new_col_names

df.head()

,dbn,school_name,borough,building_code,phone_number,fax_number,grade_span_min,grade_span_max,expgrade_span_min,expgrade_span_max,...,number_programs,location_1,community_board,council_district,census_tract,zip_codes,community_districts,borough_boundaries,city_council_districts,police_precincts
0,27Q260,Frederick Douglass Academy VI High School,Queens,Q465,718-471-2154,718-471-2890,9.0,12,NaN,NaN,...,1,"{'latitude': '40.601989336', 'longitude': '-73...",14,31,100802,20529,51,3,47,59
1,21K559,Life Academy High School for Film and Music,Brooklyn,K400,718-333-7750,718-333-7775,9.0,12,NaN,NaN,...,1,"{'latitude': '40.593593811', 'longitude': '-73...",13,47,306,17616,21,2,45,35
2,16K393,Frederick Douglass Academy IV Secondary School,Brooklyn,K026,718-574-2820,718-574-2821,9.0,12,NaN,NaN,...,1,"{'latitude': '40.692133704', 'longitude': '-73...",3,36,291,18181,69,2,49,52
3,08X305,Pablo Neruda Academy,Bronx,X450,718-824-1682,718-824-1663,9.0,12,NaN,NaN,...,1,"{'latitude': '40.822303765', 'longitude': '-73...",9,18,16,11611,58,5,31,26
4,03M485,Fiorello H. LaGuardia High School of Music & A...,Manhattan,M485,212-496-0700,212-724-5748,9.0,12,NaN,NaN,...,6,"{'latitude': '40.773670507', 'longitude': '-73...",7,6,151,12420,20,4,19,12


In [31]:
def clean_column_names(df):
    #df = pd.read_sql(query, conn)
    new_col_names=[]
    for col in df.columns:
        stripped_name=col.lower().strip().replace(' ','_')
        new_col_names.append(stripped_name)
    df.columns=new_col_names
    return df
query=f"""
SELECT * FROM nyc_schools.high_school_directory;"""
df = pd.read_sql(query, conn)
df = clean_column_names(df)
df.head()

,dbn,school_name,borough,building_code,phone_number,fax_number,grade_span_min,grade_span_max,expgrade_span_min,expgrade_span_max,...,number_programs,location_1,community_board,council_district,census_tract,zip_codes,community_districts,borough_boundaries,city_council_districts,police_precincts
0,27Q260,Frederick Douglass Academy VI High School,Queens,Q465,718-471-2154,718-471-2890,9.0,12,NaN,NaN,...,1,"{'latitude': '40.601989336', 'longitude': '-73...",14,31,100802,20529,51,3,47,59
1,21K559,Life Academy High School for Film and Music,Brooklyn,K400,718-333-7750,718-333-7775,9.0,12,NaN,NaN,...,1,"{'latitude': '40.593593811', 'longitude': '-73...",13,47,306,17616,21,2,45,35
2,16K393,Frederick Douglass Academy IV Secondary School,Brooklyn,K026,718-574-2820,718-574-2821,9.0,12,NaN,NaN,...,1,"{'latitude': '40.692133704', 'longitude': '-73...",3,36,291,18181,69,2,49,52
3,08X305,Pablo Neruda Academy,Bronx,X450,718-824-1682,718-824-1663,9.0,12,NaN,NaN,...,1,"{'latitude': '40.822303765', 'longitude': '-73...",9,18,16,11611,58,5,31,26
4,03M485,Fiorello H. LaGuardia High School of Music & A...,Manhattan,M485,212-496-0700,212-724-5748,9.0,12,NaN,NaN,...,6,"{'latitude': '40.773670507', 'longitude': '-73...",7,6,151,12420,20,4,19,12


In [22]:
borough_tuple=('Bronx','Queens')
borough_tuple

('Bronx', 'Queens')

In [44]:
query = "SELECT * FROM nyc_schools.high_school_directory LIMIT 1;"
df = pd.read_sql(query, conn)
df = clean_column_names(df)
print(df.columns.tolist())


['dbn', 'school_name', 'borough', 'building_code', 'phone_number', 'fax_number', 'grade_span_min', 'grade_span_max', 'expgrade_span_min', 'expgrade_span_max', 'start_time', 'end_time', 'priority01', 'priority02', 'priority03', 'priority04', 'priority05', 'priority06', 'priority07', 'priority08', 'priority09', 'priority10', 'location', 'phone_number2', 'school_email', 'website', 'subway', 'bus', 'grades2018', 'finalgrades', 'total_students', 'extracurricular_activities', 'school_sports', 'attendance_rate', 'pct_stu_enough_variety', 'pct_stu_safe', 'school_accessibility_description', 'directions1', 'requirement1', 'requirement2', 'requirement3', 'requirement4', 'requirement5', 'program1', 'code1', 'interest1', 'method1', 'seats9ge1', 'grade9gefilledflag1', 'grade9geapplicants1', 'seats9swd1', 'grade9swdfilledflag1', 'grade9swdapplicants1', 'campus_name', 'building_borough', 'building_location', 'latitude', 'longitude', 'community_board', 'council_district', 'census_tract', 'bin', 'bbl', 

In [32]:
query = f"""
SELECT *
FROM nyc_schools.high_school_directory 
where borough in {(borough_tuple)};"""
df = pd.read_sql(query, conn)
df = clean_column_names(df)
df.head()

,dbn,school_name,borough,building_code,phone_number,fax_number,grade_span_min,grade_span_max,expgrade_span_min,expgrade_span_max,...,number_programs,location_1,community_board,council_district,census_tract,zip_codes,community_districts,borough_boundaries,city_council_districts,police_precincts
0,27Q260,Frederick Douglass Academy VI High School,Queens,Q465,718-471-2154,718-471-2890,9.0,12,NaN,NaN,...,1,"{'latitude': '40.601989336', 'longitude': '-73...",14,31,100802,20529,51,3,47,59
1,08X305,Pablo Neruda Academy,Bronx,X450,718-824-1682,718-824-1663,9.0,12,NaN,NaN,...,1,"{'latitude': '40.822303765', 'longitude': '-73...",9,18,16,11611,58,5,31,26
2,11X509,High School of Language and Innovation,Bronx,X415,718-944-3625,718-944-3641,9.0,12,NaN,NaN,...,1,"{'latitude': '40.859698316', 'longitude': '-73...",11,13,324,11607,59,5,12,32
3,08X348,Schuylerville Preparatory High School,Bronx,X405,718-904-4200,718-935-4209,9.0,11,9.0,12.0,...,1,"{'latitude': '40.840513977', 'longitude': '-73...",10,13,194,11270,43,5,12,28
4,24Q236,International High School for Health Sciences,Queens,Q455,718-595-8600,718-595-8605,9.0,11,9.0,12.0,...,1,"{'latitude': '40.7412052', 'longitude': '-73.8...",4,25,461,14784,66,3,5,68


In [133]:
#How many schools are there in each borough?

query=f""" SELECT borough,count(*) as school_count 
FROM nyc_schools.high_school_directory 
GROUP BY borough
ORDER BY school_count desc ;"""
df = pd.read_sql(query, conn)
df = clean_column_names(df)
print(df)

         borough  school_count
0       Brooklyn           121
1          Bronx           118
2      Manhattan           106
3         Queens            80
4  Staten Island            10


In [45]:
query=f""" select DISTINCT ell_programs  FROM nyc_schools.high_school_directory 
 ;"""
df = pd.read_sql(query, conn)
df = clean_column_names(df)
print(df)

                                        ell_programs
0  ESL; Dual Language: Chinese; Transitional Bili...
1                                                ESL
2                        ESL; Dual Language: Spanish
3  ESL; Transitional Bilingual Program: Bengali, ...
4  ESL; Transitional Bilingual Program: Chinese, ...
5       ESL; Transitional Bilingual Program: Spanish
6  ESL; Dual Language: Spanish; Transitional Bili...
7  ESL; Transitional Bilingual Program: Arabic, C...
8  ESL; Dual Language: Haitian Creole, Russian; T...
9       ESL; Transitional Bilingual Program: Chinese


In [48]:
query=f"""
SELECT * FROM nyc_schools.school_demographics;"""
df = pd.read_sql(query, conn)
df

,dbn,Name,schoolyear,fl_percent,frl_percent,total_enrollment,prek,k,grade1,grade2,...,black_num,black_per,hispanic_num,hispanic_per,white_num,white_per,male_num,male_per,female_num,female_per
0,01M015,P.S. 015 ROBERTO CLEMENTE,20052006,89.4,NaN,281,15,36,40,33,...,74,26.3,189,67.3,5,1.8,158,56.2,123,43.8
1,01M015,P.S. 015 ROBERTO CLEMENTE,20062007,89.4,NaN,243,15,29,39,38,...,68,28.0,153,63.0,4,1.6,140,57.6,103,42.4
2,01M015,P.S. 015 ROBERTO CLEMENTE,20072008,89.4,NaN,261,18,43,39,36,...,77,29.5,157,60.2,7,2.7,143,54.8,118,45.2
3,01M015,P.S. 015 ROBERTO CLEMENTE,20082009,89.4,NaN,252,17,37,44,32,...,75,29.8,149,59.1,7,2.8,149,59.1,103,40.9
4,01M015,P.S. 015 ROBERTO CLEMENTE,20092010,,96.5,208,16,40,28,32,...,67,32.2,118,56.7,6,2.9,124,59.6,84,40.4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
238,01M064,PS 064 ROBERT SIMON,20072008,79.3,NaN,295,26,48,49,47,...,70,23.7,206,69.8,5,1.7,149,50.5,146,49.5
239,01M064,PS 064 ROBERT SIMON,20082009,79.3,NaN,300,26,44,48,53,...,72,24.0,200,66.7,5,1.7,162,54.0,138,46.0
240,01M064,PS 064 ROBERT SIMON,20092010,,90.5,292,26,46,43,44,...,67,22.9,192,65.8,9,3.1,152,52.1,140,47.9
241,01M064,PS 064 ROBERT SIMON,20102011,,90.5,322,32,53,47,45,...,79,24.5,200,62.1,14,4.3,179,55.6,143,44.4


In [49]:
query = "SELECT * FROM nyc_schools.school_demographics LIMIT 1;"
df = pd.read_sql(query, conn)

print(df.columns.tolist())

['dbn', 'Name', 'schoolyear', 'fl_percent', 'frl_percent', 'total_enrollment', 'prek', 'k', 'grade1', 'grade2', 'grade3', 'grade4', 'grade5', 'grade6', 'grade7', 'grade8', 'grade9', 'grade10', 'grade11', 'grade12', 'ell_num', 'ell_percent', 'sped_num', 'sped_percent', 'ctt_num', 'selfcontained_num', 'asian_num', 'asian_per', 'black_num', 'black_per', 'hispanic_num', 'hispanic_per', 'white_num', 'white_per', 'male_num', 'male_per', 'female_num', 'female_per']


In [144]:
# What is the average % of English Language Learners (ELL) per borough?

query=f""" SELECT hd.borough,AVG(sd.ell_percent) as avg_ell_percent  
FROM nyc_schools.high_school_directory  hd
LEFT JOIN nyc_schools.school_demographics sd
    ON hd.dbn = sd.dbn
GROUP BY hd.borough
ORDER BY avg_ell_percent;"""
df = pd.read_sql(query, conn)
df = clean_column_names(df)
print(df)

         borough  avg_ell_percent
0      Manhattan           7.5725
1       Brooklyn              NaN
2         Queens              NaN
3  Staten Island              NaN
4          Bronx              NaN


In [67]:
query=f"""SELECT  COUNT(*) AS total_rows,
       COUNT(ell_percent) AS ell_percent_non_null
FROM nyc_schools.school_demographics
;"""
df = pd.read_sql(query, conn)

print(df)


   total_rows  ell_percent_non_null
0         243                   243


In [72]:
query=f""" SELECT hd.borough, COUNT(*) AS count_matches
FROM nyc_schools.school_demographics sd
JOIN nyc_schools.high_school_directory hd ON sd.dbn = hd.dbn
GROUP BY hd.borough
ORDER BY count_matches DESC;"""
df = pd.read_sql(query, conn)

print(df)


     borough  count_matches
0  Manhattan             40


In [127]:
# write a query to find the top 3 schools in each borough with the highest percentage of special education students (sped_percent)
query = f"""
-- Get the highest sped_percent value for each school (dbn) 
WITH per_school_max AS (
    SELECT 
        sd.dbn,
        MAX(sd.sped_percent) AS max_sped_percent
    FROM nyc_schools.school_demographics sd
    GROUP BY sd.dbn
),

-- Join the max sped_percent data with the high school directory to get school details

merged_tables AS (
    SELECT 
        hd.borough,
        hd.school_name,
        p.max_sped_percent
    FROM per_school_max p
    JOIN nyc_schools.high_school_directory hd ON p.dbn = hd.dbn
),

-- Rank the schools within each borough by descending sped_percent
ranked_schools AS (
    SELECT *,
        RANK() OVER (PARTITION BY borough ORDER BY max_sped_percent DESC) AS rank
    FROM merged_tables
)

-- Select only the top 3 schools per borough with the highest sped_percent
SELECT 
    borough, 
    school_name, 
    max_sped_percent AS sped_percent
FROM ranked_schools
WHERE rank <= 3
ORDER BY borough, rank;
"""
df = pd.read_sql(query, conn)
print(df)



     borough                                    school_name  sped_percent
0  Manhattan                     East Side Community School          28.8
1  Manhattan                        Marta Valle High School          25.9
2  Manhattan  Henry Street School for International Studies          25.1


Insights:

1.Brooklyn and the Bronx host the most schools, while Staten Island has the fewest.

2. Only Manhattan has reported a valid average percentage of English Language Learners (ELL).

Manhattan – 7.57%

Data for Brooklyn, Queens, Bronx, and Staten Island is currently unavailable or missing in the dataset.


3.Manhattan schools have consistently served a high proportion of students in special education programs.

